# 4. Network Architecture & Training

Welcome to the fourth notebook of our six part series part of our tutorial on Deep Learning for Human Activity Recognition. Within the last notebook you learned:

- What are common evaluation metrics when evaluating the performance of an Human Activity Recognition model?
- How are they defined? How are they computed? How do they differ from each other?

This notebook will teach you everything you need to know about how neural networks are defined and trained using [PyTorch](https://pytorch.org/). As mentioned during the [theoretical part](https://https://mariusbock.github.io/dl-for-har/) of this session, we will not go into detail about each building block of a neural network and how a network is trained, but rather stick to a basic level of understanding. If you want to dig deeper, we recommend you checking out other sources, like [Coursera](https://www.coursera.org/courses?query=deep%20learning) and [YouTube](https://www.youtube.com/results?search_query=deep+learning), as there are plenty of well written tutorials on the fundamentals of Deep Learning. After working through this notebook you will be able to answer the following questions:

- How do I define a sample neural network architecture in PyTorch? 
- What additional preprocessing do I need to apply to my data to fed it into my network?
- How do I define a train loop which trains my neural network?

## 4.1. Important Remarks

If you are accessing this tutorial via [Google Colab](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/training.ipynb), first make sure to use Google Colab in English. This will help us to better assist you with issues that might arise during the tutorial. There are two ways to change the default language if it isn't English already:
1. On Google Colab, go to `Help` -> `View in English` 
2. Change the default language of your browser to `English`.

To also ease the communication when communicating errors, enable line numbers within the settings of Colab.

1. On Google Colab, go to `Tools` -> `Settings` -> `Editor` -> `Show line numbers`

In general, we strongly advise you to use Google Colab as it provides you with a working Python distribution as well as free GPU resources. To make Colab use GPUs, you need to change the current notebooks runtime type via:

- `Runtime` -> `Change runtime type` -> `Dropdown` -> `GPU` -> `Save`

**Hint:** you can auto-complete code in Colab via `ctrl` + `spacebar`

For the live tutorial, we require all participants to use Colab. If you decide to rerun the tutorial at later points and rather want to have it run locally on your machine, feel free to clone our [GitHub repository](https://github.com/mariusbock/dl-for-har).

To get started with this notebook, you need to first run the code cell below. Please set `use_colab` to be `True` if you are accessing this notebook via Colab. If not, please set it to `False`. This code cell will make sure that imports from our GitHub repository will work.

In [ ]:
import os, sys

use_colab = True

module_path = os.path.abspath(os.path.join('..'))

if use_colab:
    # move to content directory and remove directory for a clean start 
    %cd /content/         
    %rm -rf dl-for-har
    # clone package repository (will throw error if already cloned)
    !git clone https://github.com/mariusbock/dl-for-har.git
    # navigate to dl-for-har directory
    %cd dl-for-har/       
else:
    os.chdir(module_path)
    
# this statement is needed so that we can use the methods of the DL-ARC pipeline
if module_path not in sys.path:
    sys.path.append(module_path)

## 4.2. Defining a Network Architecture

During this tutorial we will use [PyTorch](https://pytorch.org/) as our Deep Learning framework of choice. The open source library is one of the most popular frameworks out there for applying Deep Learning. It has all the necessary building blocks found in neural networks pre-implemented as well as offers a variety of helpful functions which can be used to easily implement your first Deep Learning script with just a few lines of code.

In the following we will define our neural network architecture. Once defined we can use our previously preprocessed sensor-data to train a network which will be able to predict the type of activities being performed for a given sliding window. 

As mentioned during the introduction to this chapter, the architecture which we will used is called **DeepConvLSTM** [[1]](#1). The architecture was introduced by Francisco Javier Ordonez and Daniel Roggen in 2016 and is to this date a state-of-the-art architecture for applying Deep Learning on Human Activity Recognition. The architecture combines both convolutional and recurrent layers.

The architecture is made of three main parts:

1. **Convolutional layers:** Convolutional layers are based on filters (e.g. a 2 by 1 matrix) shifting over some input (e.g. a sliding window) resulting in activation feature map. The main idea of convolutions is that they are able to detect a specific type of feature anywhere within the input. Within the original architecture Ordonez and Roggen apply 4 convolutional layers each with 64 filters of size 5 by 1. 
2. **LSTM layer(s):** After applying convolutional layers, Ordonez and Roggen make us of an LSTM in order to capture time dependencies on features extracted by convolutional operations. An LSTM is a type of neural network which is able to learn temporal dependencies in data via gated mechanisms. The LSTM itself is structured into layers. Within the original architecture Ordonez and Roggen employ a 2-layered LSTM with 128 hidden units. 
3. **Classification layer:** The output of the LSTM is finally fed into a classifier which is a fully-connected layer and produces the final predictions. Preceeding the classifier, Ordonez and Roggen additionally put a dropout layer, which is a form of regularization. A dropout layer randomly deactivates neurons according to a dropout probability and thus prevents the probability of your network overfitting.

Contradicting to popular belief that one needs at least a 2-layered LSTM when dealing with sequential data, within a recent work of ours, we exhibited that a 1-layered LSTM might be a better suited option when dealing with raw sensor-data [[2]](#2). Therefore, within the next code block, we will define the altered DeepConvLSTM architecture as presented in our paper which **employs a 1-layered instead of 2-layered LSTM**.

In order to give you a better idea of how to define your PyTorch implementation of the DeepConvLSTM, we already defined a [PyTorch module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) called **DeepConvLSTM** for you to start out with. A PyTorch module typically consists of two main functions - the `init()` and `forward()` function. Within the former all relevant parameters and building blocks of the neural network are defined. Within the latter the parameters and building blocks are put together, i.e. the computation of the network defined. Within the next tasks you will be asked to fill in some of the missing parts of said module function.

### Task 1: Implementing the DeepConvLSTM
1. Within the `init()` function define the activation function. Use [PyTorch's implementation](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) of the ReLU activation function called `ReLU`. Set `inplace=True`. (`lines 17-18`)
2. Within the `init()` function define the four convolution layers. Use [PyTorch's implementation](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) of a 2d-convolution called `Conv2d`. Hints on the input and dimensions are given as comments within the code. The filter size should be of size (`filter_width x 1`) (`lines 20-24`)
3. Within the `init()` function define the LSTM. Use [PyTorch's implementation](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) of a LSTM called `LSTM`. Hints on the input size of the LSTM is given as comments within the code. The `hidden_size` and `num_layers` are given as attributes within the `init()` function. (`lines 26-27`)
4. Within the `init()` define the dropout layer. Use [PyTorch's implementation](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html) of a dropout layer called `Dropout`. Pass the `Dropout` object the `drop_prob` variable defined within the `init()` function (`lines 29-30`)
5. Within the `init()` define the classifier, i.e. fully connected layer. Use [PyTorch's implementation](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) of a fully-connected layer called `Linear`. (`lines 32-33`)
6. Fill in the blanks within the `forward()` function. Apply each of the building blocks you defined in the `init()` on your input `x`. (`lines 39-43, 52-53 and 58-60`)

In [ ]:
from torch import nn

class DeepConvLSTM(nn.Module):
    def __init__(self, config):
        super(DeepConvLSTM, self).__init__()
        # parameters
        self.window_size = config['window_size']
        self.drop_prob = config['drop_prob']
        self.nb_channels = config['nb_channels']
        self.nb_classes = config['nb_classes']
        self.seed = config['seed']
        self.nb_filters = config['nb_filters']
        self.filter_width = config['filter_width']
        self.nb_units_lstm = config['nb_units_lstm']
        self.nb_layers_lstm = config['nb_layers_lstm']

        # define activation function
        self.relu = nn.ReLU(inplace=True)

        # define conv layers
        self.conv1 = nn.Conv2d(1, self.nb_filters, (self.filter_width, 1))
        self.conv2 = nn.Conv2d(self.nb_filters, self.nb_filters, (self.filter_width, 1))
        self.conv3 = nn.Conv2d(self.nb_filters, self.nb_filters, (self.filter_width, 1))
        self.conv4 = nn.Conv2d(self.nb_filters, self.nb_filters, (self.filter_width, 1))
        
        # define lstm layers
        self.lstm = nn.LSTM(input_size=self.nb_filters * self.nb_channels, hidden_size=self.nb_units_lstm, num_layers=self.nb_layers_lstm)

        # define dropout layer
        self.dropout = nn.Dropout(self.drop_prob)
        
        # define classifier
        self.fc = nn.Linear(self.nb_units_lstm, self.nb_classes)

    def forward(self, x):
        # reshape data for convolutions
        x = x.view(-1, 1, self.window_size, self.nb_channels)
        
        # apply convolution and the activation function
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        
        # sets the final sequence length 
        final_seq_len = x.shape[2]
        
        # permute dimensions and reshape for LSTM
        x = x.permute(0, 2, 1, 3)
        x = x.reshape(-1, final_seq_len, self.nb_filters * self.nb_channels)

        # apply LSTM (note: it has two outputs!)
        x, _ = self.lstm(x)
            
        # reshape data for classifier
        x = x.view(-1, self.nb_units_lstm)
        
        # apply dropout and feed data through classifier
        x = self.dropout(x)
        x = self.fc(x)
        
        # reshape data and return predicted label of last sample within final sequence (determines label of window)
        out = x.view(-1, final_seq_len, self.nb_classes)
        return out[:, -1, :]

## 4.3. Preparing your data

Great, we now have a neural network defined which we can call and use for training! But, there is one essential step missing before moving on towards the training loop - your data needs to be put into the correct format (again). In addition to the preprocessing steps that you know from the previous notebook, we also need to **make sure that our dataset is using the correct data types which are compatible with a GPU**. Furthermore, our data needs to be split into a **training** and **validation** dataset. As you know from the [theoretical part of this section](https://mariusbock.github.io/dl-for-har), within Deep Learning we essentially try to approximate a function. To judge whether the parameterized function we came up with appropriately approximates such underlying function, we validate our network's perfomance on unseen data. If the algorithm still performs well, i.e. predicts the correct labels for the unseen data, we say that we have found a **generalized function**. The next notebook will cover in more detail what different validation methods exist and go into detail why we need and what common pitfalls exist.

The following task will guide you through the necessary preprocessing one needs to apply on top of the [RealWorld (HAR) dataset](https://sensor.informatik.uni-mannheim.de/#dataset_realworld). The first step of loading the data will already be filled out for you. As you can see, we used a predefined method called `load_dataset()`, which is part of the DL-ARC feature stack.

The preprocessing consists of **four essential parts**:

1. Split the data into a training and validation dataset. The validation dataset is used to gain feedback on the perfomance of the model and functions as unseen data. Results obtained on the validation dataset can be used as an indicator whether the changes you make to a network and/ or its training process are improving or worsening results.
2. Apply the sliding window approach on top of the training and validation dataset. As you learned in the previous notebook, we do not classify a single record, but a window of records. The label of the last record within a window defines the label of the window and is our ultimate goal to predict.
3. (Optional) Omit the subject identifier column.
4. Convert the two datasets into the correct data format so that they are compatible with the GPU.

### Task 2: Getting your data ready for training
1. Split the data into a train and validation dataset. The train dataset shall consist of the data of the first two subjects. The validation dataset shall be the data of the third subject. (`lines 16-19`)
2. Segment your train and validation data into windows. Instead of going back to your defined function within the last notebook, you can use our [predefined method](https://github.com/mariusbock/dl-for-har/blob/main/data_processing/sliding_window.py) which is part of the DL-ARC feature stack called `apply_sliding_window`. It is already imported for you. (`lines 26-29`)
3. (*Optional*) Omit the first feature column (subject_identifier) from the train and validation dataset. (`lines 35-36`)
4. Convert the feature columns of the train and validation to `float32` and label column to `uint8` for GPU compatibility. Use the [built-in function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) of a pandas dataframe called `astype()`. (`lines 41-43`)


In [ ]:
import pandas as pd
import numpy as np
import warnings

from data_processing.sliding_window import apply_sliding_window
from data_processing.preprocess_data import load_dataset

warnings.filterwarnings('ignore')


# data loading (we are using a predefined method called load_dataset, which is part of the DL-ARC feature stack)
X, y, num_classes, class_names, sampling_rate, has_null = load_dataset('rwhar_3sbjs', include_null=True)
# since the method returns features and labels separatley, we need to concat them
data = np.concatenate((X, y[:, None]), axis=1)

# define the train data to be all data belonging to the first two subjects
train_data = data[data[:, 0] <= 1]
# define the validation data to be all data belonging to the third subject
valid_data = data[data[:, 0] == 2]

# settings for the sliding window (change them if you want to!)
sw_length = 50
sw_unit = 'units'
sw_overlap = 50

# apply a sliding window on top of both the train and validation data; you can use our predefined method
# you can import it via from preprocessing.sliding_window import apply_sliding_window
X_train, y_train = apply_sliding_window(train_data[:, :-1], train_data[:, -1], sliding_window_size=sw_length, unit=sw_unit, sampling_rate=50, sliding_window_overlap=sw_overlap)
X_valid, y_valid = apply_sliding_window(valid_data[:, :-1], valid_data[:, -1], sliding_window_size=sw_length, unit=sw_unit, sampling_rate=50, sliding_window_overlap=sw_overlap)

print("\nShape of the train and validation datasets after splitting and windowing: ")
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

# (optional) omit the first feature column (subject_identifier) from the train and validation dataset
X_train, X_valid = X_train[:, :, 1:], X_valid[:, :, 1:]

print("\nShape of the train and validation feature dataset after splitting and windowing: ")
print(X_train.shape, X_valid.shape)

# convert the features of the train and validation to float32 and labels to uint8 for GPU compatibility 
X_train, y_train = X_train.astype(np.float32), y_train.astype(np.uint8)
X_valid, y_valid = X_valid.astype(np.float32), y_valid.astype(np.uint8)

## 4.4. Training Your Network

Since we now have brought the data into the correct format, let's train our network with it!

A typical training loop can be divided into three steps:

1. **Definition:** You define your network, optimizer and loss
2. **Training:** Iterating over the number of epochs: you chunk your training data into so-called batches and iteratively feed them through your network. After a batch has been fed through the network, you compute the loss said batch produced. Using the loss you backprogate it through the network using the optimizer which adjusts the weights accordingly. 
3. **Validation:** After you have processed your whole training dataset, you go on to validate the predictive performance of the network. To do so you again chunk your training and validation data into batches. Iterating over all batches of both all datasets, fed the batches through the trained network and obtain its predictions. **Note:** you only want to obtain predictions and not backpropagate any loss. 

The obtained predictions can now be used to calculate standard evaluation metrics such as **precision** and **recall**. Due to being limited in time we will not talk about their computation in great detail during the tutorial. Nevertheless, we created a [separate notebook](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/evaluation.ipynb) for you which covers the most essential evaluation metrics used in HAR. Feel free to work through it if you want to accustom yourself with how each of them is calculated. 

The next task will guide you through **implementing your training and validation loop**. It will again have parts missing which you need to fill out, but will already provide you with certain code segments, to ease the task and focus on the essential parts. 

### Task 3: Define your own train loop

1. You'll see that we already defined a `config` object which you can use to pass to your network. Nevertheless, there are three values missing, namely the `window_size`, `nb_channels` and `nb_classes`. Define them correctly. (`lines 32-38`)
2. Define your DeepConvLSTM network by calling the object we previously defined. Also define the `optimizer` being the [Adam optimizer](https://pytorch.org/docs/stable/optim.html) and `criterion` being the [Cross-Entropy Loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) (`lines 40-41 and 47-49`)
3. Define the `DataLoader` objects. The `DataLoader` objects only work with [PyTorch tensor datasets](https://pytorch.org/docs/stable/data.html), which we already defined for you as the `val_dataset` and `train_dataset`  variables. Pass the `DataLoader` object the dataset variables, the `batch_size` you want to use and set `shuffle=True`. (`lines 48-54`)
4. Further define the training loop by iterating over the training `DataLoader` object. We already defined parts for you. In a nutshell: for each batch, compute the loss by passing it through the network; backprogate the computed loss using your optimizer object. Use the [.backward()](https://pytorch.org/docs/stable/autograd.html) of the loss object and [.step()](https://pytorch.org/docs/stable/optim.html) of the optimizer to do so. (`lines 74-75 and 80-83`)
5. While training obtain predictions for the train dataset. To do so obtain the final predicitons for each batch by applying the PyTorch `softmax` [function](https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html) on top of the network output. (`lines 85-86`)
6. After training obtain predictions for the validation dataset using the resulting trained network of the current epoch. Iterate again over the validation `DataLoader` object and fed each batch through the network. In addition to calculating the [cross-entropy loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), obtain the final predicitons for each batch by applying the PyTorch `softmax` [function](https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html) on top of the network output. Using the predictions the script will calculate [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html), [precision](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html), [recall](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html) and [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) on both your training and validation set. (`lines 118-119 and 124-125`)
7. Play around with different values for the parameters within the `config` file. How does each one of them influence your training loop? Feel free to also use a completly different optimizer - you can find all the different options on the [PyTorch website](https://pytorch.org/docs/stable/optim.html). (`lines 12-27`)

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import time
from misc.torchutils import seed_torch


# this is the config object which contains all relevant settings. Feel free to change them and see how it influences
# your results. Parameters which shouldn't be changed are marked.
config = {
    'nb_filters': 64,
    'filter_width': 11,
    'nb_units_lstm': 128,
    'nb_layers_lstm': 1,
    'drop_prob': 0.5,
    'seed': 1,
    'epochs': 20,
    'batch_size': 100,
    'learning_rate': 1e-4,
    'weight_decay': 1e-6,
    'gpu_name': 'cuda:0',
    'print_counts': False
}

# in order to get reproducible results, we need to seed torch and other random parts of our implementation
seed_torch(config['seed'])

# define the missing parameters within the config file. 
# window_size = size of the sliding window in units
# nb_channels = number of feature channels
# nb_classes = number of classes that can be predicted
config['window_size'] = X_train.shape[1]
config['nb_channels'] = X_train.shape[2]
config['nb_classes'] = len(class_names)

# initialize your DeepConvLSTM object 
network = DeepConvLSTM(config)

# sends network to the GPU and sets it to training mode
network.to(config['gpu_name'])
network.train()

# initialize the optimizer and loss
optimizer = torch.optim.Adam(network.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
criterion = nn.CrossEntropyLoss()

# initializes the train and validation dataset in Torch format
train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
val_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_valid), torch.from_numpy(y_valid))
    
# define the train- and valloader; use from torch.utils.data import DataLoader
trainloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)    
valloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=True)

# define your training loop; iterates over the number of epochs
for e in range(config['epochs']):
    # helper objects needed for proper documentation
    train_losses = []
    train_preds = []
    train_gt = []
    start_time = time.time()
    batch_num = 1

    # iterate over the trainloader object (it'll return batches which you can use)
    for i, (x, y) in enumerate(trainloader):
        # sends batch x and y to the GPU
        inputs, targets = x.to(config['gpu_name']), y.to(config['gpu_name'])
        optimizer.zero_grad()

        # send inputs through network to get predictions
        train_output = network(inputs)

        # calculates loss
        loss = criterion(train_output, targets.long())

        # backprogate your computed loss through the network
        # use the .backward() and .step() function on your loss and optimizer
        loss.backward()
        optimizer.step()

        # calculate actual predictions (i.e. softmax probabilites); use torch.nn.functional.softmax()
        train_output = torch.nn.functional.softmax(train_output, dim=1)

        # appends the computed batch loss to list
        train_losses.append(loss.item())

        # creates predictions and true labels; appends them to the final lists
        y_preds = np.argmax(train_output.cpu().detach().numpy(), axis=-1)
        y_true = targets.cpu().numpy().flatten()
        train_preds = np.concatenate((np.array(train_preds, int), np.array(y_preds, int)))
        train_gt = np.concatenate((np.array(train_gt, int), np.array(y_true, int)))

        # prints out every 100 batches information about the current loss and time per batch
        if batch_num % 100 == 0 and batch_num > 0:
            cur_loss = np.mean(train_losses)
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d} batches | ms/batch {:5.2f} | train loss {:5.2f}'.format(e, batch_num, elapsed * 1000 / config['batch_size'], cur_loss))
            start_time = time.time()
            batch_num += 1

    # helper objects
    val_preds = []
    val_gt = []
    val_losses = []

    # sets network to eval mode and 
    network.eval()
    with torch.no_grad():
        # iterate over the valloader object (it'll return batches which you can use)
        for i, (x, y) in enumerate(valloader):
            # sends batch x and y to the GPU
            inputs, targets = x.to(config['gpu_name']), y.to(config['gpu_name'])

            # send inputs through network to get predictions
            val_output = network(inputs)

            # calculates loss by passing criterion both predictions and true labels 
            val_loss = criterion(val_output, targets.long())

            # calculate actual predictions (i.e. softmax probabilites); use torch.nn.functional.softmax() on dim=1
            val_output = torch.nn.functional.softmax(val_output, dim=1)

            # appends validation loss to list
            val_losses.append(val_loss.item())

            # creates predictions and true labels; appends them to the final lists
            y_preds = np.argmax(val_output.cpu().numpy(), axis=-1)
            y_true = targets.cpu().numpy().flatten()
            val_preds = np.concatenate((np.array(val_preds, int), np.array(y_preds, int)))
            val_gt = np.concatenate((np.array(val_gt, int), np.array(y_true, int)))

        # print epoch evaluation results for train and validation dataset
        print("\nEPOCH: {}/{}".format(e + 1, config['epochs']),
                  "\nTrain Loss: {:.4f}".format(np.mean(train_losses)),
                  "Train Acc: {:.4f}".format(jaccard_score(train_gt, train_preds, average='macro')),
                  "Train Prec: {:.4f}".format(precision_score(train_gt, train_preds, average='macro')),
                  "Train Rcll: {:.4f}".format(recall_score(train_gt, train_preds, average='macro')),
                  "Train F1: {:.4f}".format(f1_score(train_gt, train_preds, average='macro')),
                  "\nVal Loss: {:.4f}".format(np.mean(val_losses)),
                  "Val Acc: {:.4f}".format(jaccard_score(val_gt, val_preds, average='macro')),
                  "Val Prec: {:.4f}".format(precision_score(val_gt, val_preds, average='macro')),
                  "Val Rcll: {:.4f}".format(recall_score(val_gt, val_preds, average='macro')),
                  "Val F1: {:.4f}".format(f1_score(val_gt, val_preds, average='macro')))

        # if chosen, print the value counts of the predicted labels for train and validation dataset
        if config['print_counts']:
            print('Predicted Train Labels: ')
            print(np.vstack((np.nonzero(np.bincount(train_preds))[0], np.bincount(train_preds)[np.nonzero(np.bincount(train_preds))[0]])).T)
            print('Predicted Val Labels: ')
            print(np.vstack((np.nonzero(np.bincount(val_preds))[0], np.bincount(val_preds)[np.nonzero(np.bincount(val_preds))[0]])).T)


    # set network to train mode again
    network.train()

# References

<a id="1">[1]</a>   Francisco Javier Ordóñez and Daniel Roggen. 2016. 
Deep Convolutional and LSTM Recurrent Neural Networks for Multimodal Wearable Activity Recognition.
Sensors 16, 1 (2016).  https://doi.org/10.3390/s16010115

<a id="2">[2]</a>   Marius Bock, Alexander Hoelzemann, Michael Moeller, and Kristof Van Laerhoven. 2021. Improving Deep Learning for HAR with Shallow LSTMs. In Proceedings of the 2021 International Symposium on Wearable Computers, ISWC 2021, September 21-26, 2021, ACM. https://doi.org/10.1145/3460421.3480419